$
\newcommand{\reg}{{}^\textsuperscript{\textcircled{\textsc r}}}
\newcommand{\ml}{{\bf L}}
$
# Solving Nonlinear Equations with Iterative Methods: <br> Solvers and Examples in Julia<br>   C. T. Kelley

Copyright C. T. Kelley, 2020

To Betty Thomas Kelley

## Contents

[Preface and Software](Preface_and_Software)

- [Preface for the notebook](#Preface-for-the-notebook)

- [Preface for the print book](#Preface-for-the-print-book)

- [How to get the software](#How-to-get-the-software)

- [Software License](LICENSE-CODE.md)

- [Text License (ie the notebook)](LICENSE-TEXT.md)

[Chapter 1: Introduction](SIAMFANLCh1.ipynb)

# Preface and Software

## Preface for the notebook

This notebook is a more interactive version of the print book. I encourage you to play with the examples, 
make up your own, and explore the algorithmic parameters. I hope it's more fun to play with this than to just
read text. However, the print book is more useful when you need to look something up or want to bookmark a few
pages while working on your own projects.

I hope that the three parts of this project, print book, notebook, and the solver package make one coherent thing.    


## Preface for the print book


This small book on solvers for nonlinear equations
is a user-oriented guide to algorithms and implementation.
It is a sequel to 
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite>,
which used MATLAB® for 
the solvers and examples. This book uses Julia and adds new material
on pseudo-transient continuation, mixed-precision solvers, and
Anderson acceleration.

Its purpose is to show, via algorithms
in pseudocode, in Julia, and with several examples, how one
can choose an appropriate Newton-type method for a given problem and
write an efficient solver or apply one written by others.

This book is intended to
complement my larger book complement my larger book 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
which focuses on in-depth
treatment of convergence theory, but does not discuss the details of
solving particular problems, implementation in any particular language,
or evaluating a solver for a given problem.

The computational examples in this book were
done with Julia 1.4 on various Apple computers.
The Julia codes for the solvers are in a Julia package.
I provide a Jupyter notebook to enable the reader to run all
the examples in the book and play with them.

I have found that Julia is an excellent environment for a project
like this, which unifies a print book, a package of solvers, and an
interactive notebook. I have used all the solvers in my own research
and many of the examples come from that research.

The codes are tuned for clarity and the algorithmic parameters are
exposed so the reader can play with them. I have sacrificed a
good deal of abstraction, some generality, and a bit of performance
for that clarity.  The solvers
were designed for production work on desktop and laptop comptuters
to solve small-to medium-scale
problems having at most a a few tens of thousands of unknowns.
Very large-scale problems are best done with software
(Trilinos, PETSc, SunDials) desinged for that purpose.

We assume that the reader has a good understanding of elementary
numerical analysis at the level of \cite{atkelem} and of numerical
linear algebra at the level of 
<cite data-cite="demmel"><a href="siamfa.html#demmel">(Dem97)</cite>,
<cite data-cite="IIbook"><a href="siamfa.html#IIbook">(Ips09)</cite>, and
<cite data-cite="trefbau"><a href="siamfa.html#trefbau">(TB96)</cite>.
We will also use some deeper results from
numerical linear algebra and will refer to 
<cite data-cite="higham"><a href="siamfa.html#higham">(Hig96)</cite>
for those.

Because the examples are so closely coupled to the text,
this book cannot be understood without a working knowledge of Julia.
Unfortunately, there is no introduction to the language aimed at
the numerical analyst, as there is for Matlab 
<cite data-cite="nick_matlab"><a href="siamfa.html#nick_matlab">(HH05)</cite>.
One very good reference is the paper 
<cite data-cite="Juliasirev"><a href="siamfa.html#Juliasirev">(BEKS17)</cite>.
The full manual 
<cite data-cite="Julia20"><a href="siamfa.html#Julia20">(Pro20)</cite> and the online guide
<cite data-cite="JuliaWeb"><a href="siamfa.html#JuliaWeb">(Ame20)</cite>
also very useful.

I do not expect the reader to be an expert
in Julia, but do expect the reader to be able to understand the codes,
play with the algorithms, and wreak havoc. To that end I have tuned the
codes for readability by a Julia novice who knows some numerical analysis. I
made the algorithmic parameters very easy to find. As part of this I
have sacrificed a lot of abstraction, some generality, and a bit
of performance.
    
The reader should know Julia and its ecosystem
well enough to use the package manager,
install packages, use modules, and to do basic tasks in
numerical methods (LU, SVD, QR) in Julia. The reader should also know
how to use github to clone repositories and put
them where they need to go.

Parts of this book are based on research supported by the National
Science Foundation,
and the Army Research Office, most recently
by NSF grants
OAC-1740309,
DMS-1745654,
DMS-1906446,
and
ARO grant W911NF-16-1-0504.

Any opinions, findings, and conclusions or
recommendations expressed in this material are those of the author and
do not necessarily reflect the views of the National
Science Foundation
or the Army Research Office.

In addition to the many people who influenced
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite> and
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
I want
to particularly thank
Jerry Bernholc,
Wei Bian,
Emil Briggs,
Xiaojun Chen,
Kevin Clarno,
Austin Ellis,
Tom Evans,
Elizabeth Greenspan,
Steven Hamilton,
Nick Higham,
Ilse Ipsen,
Elena Jakubikova,
Randy LeVeque,
Lin Lin,
Juan Meza,
James Nance,
Chung-Wei Ng,
Roger Pawlowski,
Liqun Qi,
Stuart Slattery,
Alex Toth,
Homer Walker,
Jeff Willert,
Chao Yang,
and a few hundred MA 784 students
for their inspiration as I got this project into shape.
    
C. T. Kelley<br>
Raleigh, North Carolina<br>
May 2020


## How to get the software

If you are reading this, you've found the notebook and probably have installed the package. The notebooks for the individual chapters get the packages organized in the first code window, which you __must run!__

I am assuming you've followed the directions in the README for the repo and cloned the whole works to a place where Julia can find it. I will use the module /src/NotebookSIAMFANL.jl in the notebooks. That module has the examples from the book organized in a sensible way.

The project has three parts:

- this notebook
- the Julia package SIAMFANLEquations.jl
- the test problems

As you will see, that first code window in each chapter has

```julia
using SIAMFANLEquations
using SIAMFANLEquations.TestProblems
using NotebookSIAMFANL
using PyPLot
```
at the start. 
